In [ ]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [ ]:
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/'
df = pd.read_csv(PATH + 'train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

In [ ]:
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spectrogram_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values

y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head(5)

In [ ]:
EEG_PATH = PATH + 'train_eegs/'

row = train.iloc[1]
eeg_id = row['eeg_id']

eeg_file_path = os.path.join(EEG_PATH, f'{eeg_id}.parquet')
sample_eeg_data = pd.read_parquet(eeg_file_path)
sample_eeg_data = sample_eeg_data.iloc[:, :-1]
sample_eeg_data = sample_eeg_data.iloc[:, [col for col in range(sample_eeg_data.shape[1]) if col not in [8, 9, 10]]]

start_time_point = int((sample_eeg_data.shape[0] - 10_000) // 2)

eeg_slice = sample_eeg_data.iloc[start_time_point : start_time_point + 10_000, :]

eeg_slice

In [ ]:
FLAG = True

In [ ]:
if FLAG:
    fs = 200
    EEG_PATH = PATH + 'train_eegs/'
    all_eeg_data = np.zeros((len(train), 10000, 19))

    for row_idx in range(len(train)):
        print(row_idx, end = ", ")
        row = train.iloc[row_idx]
        eeg_id = row['eeg_id']

        eeg_file_path = os.path.join(EEG_PATH, f'{eeg_id}.parquet')
        eeg_data = pd.read_parquet(eeg_file_path)
        eeg_data = eeg_data.iloc[:, :-1]
        #eeg_data = eeg_data.iloc[:, [col for col in range(eeg_data.shape[1]) if col not in [8, 9, 10]]]
        start_time_point = int((eeg_data.shape[0] - 10_000) // 2)

        eeg_slice = eeg_data.iloc[start_time_point : start_time_point + 10_000, :]

        all_eeg_data[row_idx, :, :] = eeg_slice

    print("Shape of all_eeg_data:", all_eeg_data.shape)
else:
    print("FLAG is set to False. The code below is not executed.")

# Feature Extraction

In [ ]:
import scipy.stats

def extract_eeg_features(eeg_data):
    """
    Extract features from EEG data, including LL Spec, LP Spec, RP Spec, RL Spec

    Parameters:
    - eeg_data: EEG data with shape (number of samples, time points, number of electrodes)

    Returns:
    - Feature matrix with shape (number of samples, number of features)
    """
    # Initialize the feature matrix
    num_samples, num_time_points, num_electrodes = eeg_data.shape
    num_features = 8 * num_electrodes  # Mean, standard deviation, minimum, maximum, skewness, kurtosis, energy, entropy for each electrode + 4 global features
    features = np.zeros((num_samples, num_features))

    # Extract features
    for sample_idx in range(num_samples):
        for electrode_idx in range(num_electrodes):
            electrode_data = eeg_data[sample_idx, :, electrode_idx]
            feature_idx = electrode_idx * 8

            # Mean
            features[sample_idx, feature_idx] = np.mean(electrode_data)
            # Standard deviation
            features[sample_idx, feature_idx + 1] = np.std(electrode_data)
            # Minimum
            features[sample_idx, feature_idx + 2] = np.min(electrode_data)
            # Maximum
            features[sample_idx, feature_idx + 3] = np.max(electrode_data)
            # Skewness
            features[sample_idx, feature_idx + 4] = scipy.stats.skew(electrode_data)
            # Kurtosis
            features[sample_idx, feature_idx + 5] = scipy.stats.kurtosis(electrode_data)
            # Energy
            features[sample_idx, feature_idx + 6] = np.sum(electrode_data**2) / len(electrode_data)
            # Entropy
            features[sample_idx, feature_idx + 7] = scipy.stats.entropy(np.abs(electrode_data))
        # Additional features: LL Spec, LP Spec, RP Spec, RL Spec
        ll_spec = (np.abs(np.fft.fft(eeg_data[sample_idx, :, 0] - eeg_data[sample_idx, :, 4]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 4] - eeg_data[sample_idx, :, 5]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 5] - eeg_data[sample_idx, :, 6]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 6] - eeg_data[sample_idx, :, 7]))**2) / 4
        features[sample_idx, -4] = np.mean(ll_spec)

        lp_spec = (np.abs(np.fft.fft(eeg_data[sample_idx, :, 0] - eeg_data[sample_idx, :, 1]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 1] - eeg_data[sample_idx, :, 2]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 2] - eeg_data[sample_idx, :, 3]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 3] - eeg_data[sample_idx, :, 7]))**2) / 4
        features[sample_idx, -3] = np.mean(lp_spec)

        rp_spec = (np.abs(np.fft.fft(eeg_data[sample_idx, :, 8] - eeg_data[sample_idx, :, 9]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 9] - eeg_data[sample_idx, :, 10]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 10] - eeg_data[sample_idx, :, 11]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 11] - eeg_data[sample_idx, :, 15]))**2) / 4
        features[sample_idx, -2] = np.mean(rp_spec)

        rl_spec = (np.abs(np.fft.fft(eeg_data[sample_idx, :, 8] - eeg_data[sample_idx, :, 12]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 12] - eeg_data[sample_idx, :, 13]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 13] - eeg_data[sample_idx, :, 14]))**2 +
                   np.abs(np.fft.fft(eeg_data[sample_idx, :, 14] - eeg_data[sample_idx, :, 15]))**2) / 4
        features[sample_idx, -1] = np.mean(rl_spec)
    return features

In [ ]:
import numpy as np
import scipy

from scipy.stats import skew, kurtosis
from scipy.stats.mstats import moment
from sklearn.preprocessing import StandardScaler

if FLAG:
    extracted_features = extract_eeg_features(all_eeg_data)
    print("Shape of extracted features:", extracted_features.shape)
else:
    print("FLAG is set to False. The code below is not executed.")

In [ ]:
if FLAG:
    mean_values = np.nanmean(extracted_features, axis=0)
    nan_indices = np.isnan(extracted_features)
    extracted_features[nan_indices] = np.take(mean_values, nan_indices.nonzero()[1])
    print("Shape of eeg_features after replacing NaN values:", extracted_features.shape)
    save_path = '/kaggle/working/extracted_eeg_features.npy'
    np.save(save_path, extracted_features)
    print(f"Extracted EEG features saved at: {save_path}")
else:
    extracted_features = np.load('/kaggle/input/8-basic-feaatures-with-eeg/extracted_eeg_features (1).npy')
    print("Shape of extracted features:", extracted_features.shape)

In [ ]:
ef = pd.DataFrame(extracted_features)
ef

# Label

In [ ]:
train_copy = train.copy()
ycol = [c for c in train_copy.columns if c in ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']]
cd = {'Seizure':'seizure_vote', 'GPD':'gpd_vote', 'LRDA':'lrda_vote', 'Other':'other_vote', 'GRDA':'grda_vote', 'LPD':'lpd_vote'}
train_copy['target'] = train_copy['target'].map(cd)
for i in range(len(train_copy)):
    c = train_copy['target'][i]
    train_copy[c][i] = train_copy[c][i]+10 #adding weight to expert consensus

ysum = train_copy[ycol].sum(axis=1)
for c in ycol:
    train_copy[c] = (train_copy[c] / ysum).astype(np.float64)

In [ ]:
label = train_copy[ycol]
label.shape

# Model

In [ ]:
ef

In [ ]:
# Standardize the extracted features
scaler = StandardScaler()
standardized_features = scaler.fit_transform(extracted_features)

# Convert the standardized features back to a DataFrame if needed
standardized_ef = pd.DataFrame(standardized_features)

# Display the shape of standardized features
print("Shape of standardized features:", standardized_ef.shape)

In [ ]:
standardized_ef

# **CatBoost**

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X = standardized_features
y_labels = np.argmax(label.values, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y_labels, test_size=0.2, random_state=42)

from catboost import CatBoostClassifier

catboost_clf = CatBoostClassifier(iterations=1000,
                                  depth=12,
                                  learning_rate=0.05,
                                  loss_function='MultiClass',
                                  verbose=-1,
                                  task_type="GPU")

catboost_clf.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)
y_pred_catboost = catboost_clf.predict(X_test)

# Metrics
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
print(f"Accuracy (CatBoost): {accuracy_catboost}")

# Convert class labels to strings
class_names = [str(label) for label in np.unique(y_labels)]
# Confusion Matrix
conf_mat_catboost = confusion_matrix(y_test, y_pred_catboost)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_catboost, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_labels), yticklabels=np.unique(y_labels))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (CatBoost)')
plt.show()

# Classification Report
print(classification_report(y_test, y_pred_catboost, target_names=class_names))

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

# Predict probabilities
y_probs_catboost = catboost_clf.predict_proba(X_test)

# Calculate MSE and MAE
mse_catboost = mean_squared_error(y_test, y_pred_catboost)
mae_catboost = mean_absolute_error(y_test, y_pred_catboost)

# Accuracy
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)

# Precision
precision_catboost = precision_score(y_test, y_pred_catboost, average='macro')

# Recall
recall_catboost = recall_score(y_test, y_pred_catboost, average='macro')

# F1-score
f1_catboost = f1_score(y_test, y_pred_catboost, average='macro')

# Print the results
print("Metrics for CatBoost:")
print(f"Mean Squared Error (MSE): {mse_catboost}")
print(f"Mean Absolute Error (MAE): {mae_catboost}")
print(f"Accuracy: {accuracy_catboost}")
print(f"Precision: {precision_catboost}")
print(f"Recall: {recall_catboost}")
print(f"F1-score: {f1_catboost}")

In [ ]:
# Plot ROC curve for each class
fpr_catboost = dict()
tpr_catboost = dict()
roc_auc_catboost = dict()
for i in range(len(np.unique(y_labels))):
    fpr_catboost[i], tpr_catboost[i], _ = roc_curve(y_test == i, y_probs_catboost[:, i])
    roc_auc_catboost[i] = auc(fpr_catboost[i], tpr_catboost[i])

plt.figure(figsize=(10, 8))
for i in range(len(np.unique(y_labels))):
    plt.plot(fpr_catboost[i], tpr_catboost[i], label=f'Class {i} (AUC = {roc_auc_catboost[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-AUC Curve for Each Class (CatBoost)')
plt.legend(loc="lower right")
plt.show()

# Infer Test and Create Submission CSV

In [ ]:
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
print('Test shape',test.shape)
test.head()

In [ ]:
# FEATURE ENGINEER TEST
PATH2 = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/'
data_test = np.zeros((len(test), extracted_features.shape[1]))
test_eeg = np.zeros((len(test), 10000, 19))

for k in range(len(test)):
    row = test.iloc[k]
    s = int(row.eeg_id)
    eeg_test = pd.read_parquet(f'{PATH2}{s}.parquet')
    eeg_test = eeg_test.iloc[:, :-1]
    #eeg_test = eeg_test.iloc[:, [col for col in range(eeg_test.shape[1]) if col not in [8, 9, 10]]]
    start_time_point = int((eeg_test.shape[0] - 10_000) // 2)
    eeg_test_slice = eeg_test.iloc[start_time_point : start_time_point + 10_000, :]

    test_eeg[k, :, :] = eeg_test_slice
    features_test = extract_eeg_features(test_eeg)
    print("Shape of features_test:", features_test.shape)
    data_test = features_test
print("Shape of data_test:", data_test.shape)

In [ ]:
predictions_proba = catboost_clf.predict_proba(data_test)
class_names = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
probabilities_df = pd.DataFrame(predictions_proba, columns=class_names)
print(probabilities_df.head(1))

In [ ]:
sub = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub[TARGETS] = probabilities_df
sub.to_csv('submission.csv',index=False)
print('Submissionn shape',sub.shape)
sub.head()

In [ ]:
# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
sub.iloc[:,-6:].sum(axis=1)